## Installing the Dependencies

In [ ]:
# !pip install langchain
# !pip install langchain-groq
# !pip install "langserve[all]"
# !pip install langchain_community["all"]
# !pip install chromadb
# !pip install langchain-experimental

In [ ]:
from rich import print
import getpass
import os
from langchain.docstore.document import Document
from langchain_community.chat_models import ChatOllama
from langchain_community.vectorstores import Chroma
from langchain_community import embeddings
from langchain_core.runnables import RunnablePassthrough
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings
from google.colab import userdata
print("Everything Imported succesfully✅")

Everything Imported succesfully✅

In [ ]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()
os.environ["GROQ_API_KEY"] = getpass.getpass()

··········
··········


### Initializing the Model

In [ ]:
# Loading the LLM model
model = ChatGroq(model="llama3-8b-8192")

# Embedding Model
embedding_model = HuggingFaceInferenceAPIEmbeddings(api_key =userdata.get('HF_TOKEN'),model_name="BAAI/bge-base-en-v1.5")

### Defining the RAG

In [ ]:
# RAG
def rag(chunks, collection_name,embedding_model):

    # Defining the vector store
    vectorstore = Chroma.from_documents(
        documents=documents,
        collection_name=collection_name,
        embedding=embedding_model,
    )
    retriever = vectorstore.as_retriever()

    # Defining the prompt template
    prompt_template = """Answer the question based only on the following context:
    {context}
    Question: {question}
    """
    prompt = ChatPromptTemplate.from_template(prompt_template)

    # Defining the chain using LECL
    chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | prompt
        | model
        | StrOutputParser()
    )
    result = chain.invoke("What factors influence the efficiency of heat transfer in a heat exchanger?")
    print(result)

In [ ]:
text = """The primary function of a heat exchanger is to transfer heat between two or more fluids.
The efficiency of this heat transfer process depends on factors such as the surface area of the heat exchanger,
the temperature difference between the fluids, and the flow arrangement. In a shell and tube heat exchanger,
for instance, one fluid flows through the tubes while the other fluid flows around the tubes within the shell,
maximizing the heat exchange surface."""

### Character text splitting

In [ ]:
# Automatic Text Splitting
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size = 30, chunk_overlap=0, separator='', strip_whitespace=False)
documents = text_splitter.create_documents([text])
print(documents)

[
    Document(page_content='The primary function of a heat'),
    Document(page_content=' exchanger is to transfer heat'),
    Document(page_content=' between two or more fluids. \n'),
    Document(page_content='The efficiency of this heat tr'),
    Document(page_content='ansfer process depends on fact'),
    Document(page_content='ors such as the surface area o'),
    Document(page_content='f the heat exchanger, \nthe tem'),
    Document(page_content='perature difference between th'),
    Document(page_content='e fluids, and the flow arrange'),
    Document(page_content='ment. In a shell and tube heat'),
    Document(page_content=' exchanger, \nfor instance, one'),
    Document(page_content=' fluid flows through the tubes'),
    Document(page_content=' while the other fluid flows a'),
    Document(page_content='round the tubes within the she'),
    Document(page_content='ll, \nmaximizing the heat excha'),
    Document(page_content='nge surface.')
]

In [ ]:
rag(documents,"text-splitting",embedding_model)

Based on the provided context, I can answer the question:

The primary function of a heat exchanger is not explicitly stated in the provided documents. However, we can infer 
that the primary function of a heat exchanger is being discussed, as the same phrase is repeated multiple times in 
different documents.

### Recursive Character splitter

In [ ]:
# Automatic Text Splitting
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 30, chunk_overlap=0, separators=[], strip_whitespace=False)
documents = text_splitter.create_documents([text])
print(documents)

[
    Document(page_content='The primary function of a heat'),
    Document(page_content=' exchanger is to transfer heat'),
    Document(page_content=' between two or more fluids. '),
    Document(page_content='\nThe efficiency of this heat'),
    Document(page_content=' transfer process depends on'),
    Document(page_content=' factors such as the surface'),
    Document(page_content=' area of the heat exchanger, '),
    Document(page_content='\nthe temperature difference'),
    Document(page_content=' between the fluids, and the'),
    Document(page_content=' flow arrangement. In a shell'),
    Document(page_content=' and tube heat exchanger, '),
    Document(page_content='\nfor instance, one fluid flows'),
    Document(page_content=' through the tubes while the'),
    Document(page_content=' other fluid flows around the'),
    Document(page_content=' tubes within the shell, '),
    Document(page_content='\nmaximizing the heat exchange'),
    Document(page_content=' surface.')
]

In [ ]:
rag(documents,"recu-splitting",embedding_model)

Based on the given context, it is not possible to determine the exact factors that influence the efficiency of heat
transfer in a heat exchanger. The context only mentions "maximizing the heat exchange" and "area of the heat 
exchanger", but does not provide any information about the factors that affect the efficiency of heat transfer.

### Semantic Chunking

In [ ]:
from langchain_experimental.text_splitter import SemanticChunker

# Percentile - all differences between sentences are calculated, and then any difference greater than the X percentile is split
text_splitter = SemanticChunker(
    embedding_model, breakpoint_threshold_type="percentile" # "standard_deviation", "interquartile"
)
documents = text_splitter.create_documents([text])
print(documents)

[
    Document(
        page_content='The primary function of a heat exchanger is to transfer heat between two or more fluids. The 
efficiency of this heat transfer process depends on factors such as the surface area of the heat exchanger, \nthe 
temperature difference between the fluids, and the flow arrangement.'
    ),
    Document(
        page_content='In a shell and tube heat exchanger, \nfor instance, one fluid flows through the tubes while 
the other fluid flows around the tubes within the shell, \nmaximizing the heat exchange surface.'
    )
]

In [ ]:
rag(documents,"semantic-splitting",embedding_model)

According to the context, the factors that influence the efficiency of heat transfer in a heat exchanger are:

1. Surface area of the heat exchanger
2. Temperature difference between the fluids
3. Flow arrangement